# Alexander Toledo
## Analisis Exploratorio de Datos (EDA)
### Dataset: Customer Support on Twitter

In [1]:
import pandas as pd
import numpy as np

In [4]:
# Guardamos la data en un df
df = pd.read_csv('twcs.csv')
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


## Contexto de las variables
### tweet_id
- A unique, anonymized ID for the Tweet. Referenced by response_tweet_id and in_response_to_tweet_id.
- Identificador único y anonimizado del tweet. Es referenciado por las variables response_tweet_id y in_response_to_tweet_id.
### author_id
- A unique, anonymized user ID. @s in the dataset have been replaced with their associated anonymized user ID.
- Identificador único y anonimizado del usuario. Las menciones (@) en el dataset fueron reemplazadas por su identificador de usuario anonimizado. 
### inbound
- Whether the tweet is "inbound" to a company doing customer support on Twitter. This feature is useful when re-organizing data for training conversational models.
- Indica si el tweet es una interacción “entrante” hacia una empresa que realiza atención al cliente en Twitter. Esta variable es útil para reorganizar los datos, por ejemplo, al entrenar modelos conversacionales.
### created_at
- Date and time when the tweet was sent.
- Fecha y Hora de cuando el tweet fue enviado.
### text
- Tweet content. Sensitive information like phone numbers and email addresses are replaced with mask values like "__email __".
- Contenido del tweet.
Información sensible como números de teléfono o correos electrónicos fue reemplazada por valores enmascarados, como "__email __". 
### response_tweet_id
- IDs of tweets that are responses to this tweet, comma-separated.
- Identificadores de los tweets que son respuestas a este tweet, separados por comas.
### in_response_to_tweet_id
- ID of the tweet this tweet is in response to, if any.
- Identificador del tweet al cual este tweet responde, si corresponde.